In [ ]:
# -*- coding: utf-8 -*-
"""
Molecular Data Machine Learning: Exploring Predictive Models

This notebook explores the "Molecular Data Machine Learning" Kaggle competition,
focusing on predicting molecular properties (coupling constants). We'll load
and examine the data, perform feature engineering, visualize relationships,
and train several regression models, including linear models, tree-based
methods, and ensemble techniques. The goal is to build a robust and accurate
predictive model, keeping in mind the dataset's characteristics of having
relatively few rows but many features.

Author: AI Assistant (based on user's request)
"""

# ## 1. Importing Necessary Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import lightgbm as lgb  # You might need to install this: pip install lightgbm
import xgboost as xgb  # You might need to install this: pip install xgboost
from rdkit import Chem #You might need to install this: conda install -c conda-forge rdkit

# ## 2. Loading the Data

# Load the training data. For demonstration purposes, we'll assume the data
# is in 'train.csv'. You would replace this with the actual path to the training data.
try:
    train_df = pd.read_csv('train.csv')
    print("Training data loaded successfully.")
    print(f"Shape of training data: {train_df.shape}")
except FileNotFoundError:
    print("Error: train.csv not found. Please ensure the file is in the correct directory.")
    # For demonstration, let's create a dummy DataFrame with the discussed structure
    n_rows = 100  # Simulate 'few rows'
    n_cols = 50  # Simulate 'many columns'
    dummy_data = np.random.rand(n_rows, n_cols)
    dummy_cols = [f'feature_{i}' for i in range(n_cols)] + ['target', 'molecule_structure'] # Added dummy column
    train_df = pd.DataFrame(np.hstack((dummy_data, np.random.rand(n_rows, 2))), columns=dummy_cols)
    train_df['molecule_structure'] = ['C1CCCCC1', 'C1=CC=CC=C1', 'CC(=O)C', 'C1CN2C(=O)C(=O)CC2', 'C1OC2OC(OC(C2C1O)n3c4ccccc4c5nc[nH]c3=O)C(O)(C(C(C6=O)OCC7OC(F)(F)C(F)(F)C(F)(F)C7F)(F)F)C(F)(F)F',
                            'C1=CC2C(=CC(=O)C(=C2C(=O)O1)C3=CC=CC=C3)C4=CC=CC=C4',
                            'C1CCCCCC1', 'C1=CC=CC=C1', 'CC(=O)C', 'C1CN2C(=O)C(=O)CC2'] * 10
    print("Using a dummy training dataset for demonstration.")

# Assuming 'target' is the column we want to predict (replace with the actual target column name)
TARGET_COL = 'target'
FEATURES = [col for col in train_df.columns if col != TARGET_COL and col != 'molecule_structure'] # Exclude the molecule structure
MOLECULE_COL = 'molecule_structure' # Define new variable

# ## 3. Exploratory Data Analysis (EDA) - A Glimpse

# Let's get a quick overview of the data.
print("\nFirst few rows of the training data:")
print(train_df.head())

print("\nSummary statistics of the training data:")
print(train_df.describe())

# Visualize the distribution of the target variable.
plt.figure(figsize=(8, 6))
sns.histplot(train_df[TARGET_COL], kde=True)
plt.title(f'Distribution of {TARGET_COL}')
plt.xlabel(TARGET_COL)
plt.ylabel('Frequency')
plt.show()

# Visualize the correlation matrix of a subset of features (to avoid overwhelming visualization).
# This can help identify relationships between features.
if len(FEATURES) > 10:
    subset_cols = FEATURES[:10] + [TARGET_COL]
else:
    subset_cols = FEATURES + [TARGET_COL]

correlation_matrix = train_df[subset_cols].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')
plt.title('Correlation Matrix of a Subset of Features')
plt.show()

# Scatter plot of a few features against the target variable to visualize relationships.
if len(FEATURES) >= 2:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    sns.scatterplot(x=train_df[FEATURES[0]], y=train_df[TARGET_COL], ax=axes[0])
    axes[0].set_title(f'{FEATURES[0]} vs {TARGET_COL}')
    sns.scatterplot(x=train_df[FEATURES[1]], y=train_df[TARGET_COL], ax=axes[1])
    axes[1].set_title(f'{FEATURES[1]} vs {TARGET_COL}')
    plt.tight_layout()
    plt.show()

# Visualize a few molecules from the  column
if MOLECULE_COL in train_df.columns:
    molecules = train_df[MOLECULE_COL].unique()[:4]  # Show a maximum of 4 molecules
    fig, axes = plt.subplots(1, len(molecules), figsize=(15, 5))
    for i, mol_smiles in enumerate(molecules):
        try:
            mol = Chem.MolFromSmiles(mol_smiles) # Convert SMILES to Molecule object
            if mol:
                Chem.Draw.MolToImage(mol, size=(200, 200), ax=axes[i], fitImage=True)
                axes[i].set_title(f'Molecule {i+1}')
            else:
                axes[i].text(0.5, 0.5, f'Invalid SMILES: {mol_smiles}', ha='center', va='center', transform=axes[i].transAxes)
        except Exception as e:
             axes[i].text(0.5, 0.5, f'Error: {e}', ha='center', va='center', transform=axes[i].transAxes)
    plt.tight_layout()
    plt.show()

# ## 4. Feature Engineering

# Example of a simple feature engineering step: creating polynomial features.
# In a real-world scenario, this is where you'd use RDKit to generate
# molecular descriptors.  Since the dummy data does not have molecular information,
# we will skip RDKit.
try:
  poly = PolynomialFeatures(degree=2, include_bias=False)
  X_poly = poly.fit_transform(train_df[FEATURES])
  X_poly_df = pd.DataFrame(X_poly, columns=[f'poly_{i}' for i in range(X_poly.shape[1])])
  train_df = pd.concat([train_df, X_poly_df], axis=1)
  FEATURES = FEATURES + [col for col in train_df.columns if 'poly_' in col]
  print("Polynomial features created.")
except Exception as e:
  print(f"Error creating polynomial features: {e}")

# ## 5. Data Preprocessing

# Separate features (X) and target (y).
X = train_df[FEATURES]
y = train_df[TARGET_COL]

# Split the data into training and testing sets for initial evaluation.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features. This is important for linear models and some tree-based models.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled data back to DataFrames for easier handling later if needed.
X_train_scaled = pd.DataFrame(X_train_scaled, columns=FEATURES)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=FEATURES)

# ## 6. Model Training and Evaluation

# We will train and evaluate several regression models using cross-validation
# to get a more robust estimate of their performance on unseen data.
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scoring_metric = 'neg_mean_absolute_error' # Using negative MAE as cross_val_score maximizes

def evaluate_model(model, X, y, cv, scoring):
    scores = cross_val_score(model, X, y, cv=cv, scoring=scoring)
    mae_scores = -scores # Convert negative MAE back to positive
    print(f"{type(model).__name__}: Mean MAE = {mae_scores.mean():.4f}, Std MAE = {mae_scores.std():.4f}")
    return mae_scores.mean()

# ### 6.1. Linear Regression

linear_reg = LinearRegression()
evaluate_model(linear_reg, X_train_scaled, y_train, cv, scoring_metric)

# ### 6.2. Ridge Regression (L2 Regularization)

ridge_reg = Ridge(alpha=1.0) # You can tune alpha
evaluate_model(ridge_reg, X_train_scaled, y_train, cv, scoring_metric)

# ### 6.3. Lasso Regression (L1 Regularization)

lasso_reg = Lasso(alpha=0.01) # You can tune alpha
evaluate_model(lasso_reg, X_train_scaled, y_train, cv, scoring_metric)

# ### 6.4. Polynomial Regression (to capture non-linear relationships)

poly_pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('linear', LinearRegression())
])
evaluate_model(poly_pipeline, X_train, y_train, cv, scoring_metric)

# ### 6.5. Random Forest Regressor

rf_reg = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
evaluate_model(rf_reg, X_train, y_train, cv, scoring_metric)

# ### 6.6. Gradient Boosting Regressor

gb_reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
evaluate_model(gb_reg, X_train, y_train, cv, scoring_metric)

# ### 6.7. LightGBM Regressor

lgbm_reg = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42, n_jobs=-1)
evaluate_model(lgbm_reg, X_train, y_train, cv, scoring_metric)

# ### 6.8. XGBoost Regressor

xgb_reg = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42, n_jobs=-1)
evaluate_model(xgb_reg, X_train, y_train, cv, scoring_metric)

# ### 6.9. Voting Regressor (Ensemble of diverse models)

voting_reg = VotingRegressor(estimators=[
    ('lr', linear_reg),
    ('ridge', ridge_reg),
    ('lasso', lasso_reg),
    ('rf', rf_reg),
    ('gb', gb_reg),
    ('lgbm', lgbm_reg),
    ('xgb', xgb_reg)
])
evaluate_model(voting_reg, X_train_scaled, y_train, cv, scoring_metric)

# ## 7. Model Evaluation on the Test Set

# Let's evaluate the performance of one of the promising models (e.g., Gradient Boosting)
# on the held-out test set.

final_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
final_model.fit(X_train, y_train)
predictions = final_model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f"\nPerformance of Gradient Boosting Regressor on the Test Set: MAE = {mae:.4f}")

# Visualize the predictions against the actual values on the test set.
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=predictions)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values (Gradient Boosting on Test Set)')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2) # Diagonal line for perfect predictions
plt.show()

# Visualize the residuals (the difference between actual and predicted values).
residuals = y_test - predictions
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals (Gradient Boosting on Test Set)')
plt.xlabel('Residuals (Actual - Predicted)')
plt.ylabel('Frequency')
plt.show()

# ## 8. Feature Importance

# Display feature importance for a tree-based model (e.g., Gradient Boosting)
if hasattr(final_model, 'feature_importances_'):
    feature_importance = final_model.feature_importances_
    feature_names = X_train.columns
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df)
    plt.title('Feature Importance (Gradient Boosting)')
    plt.show()
else:
    print("\nFeature importance is not available for the final model.")

# ## 9. Analysis of Errors
# Get more detail on model errors
def analyze_errors(y_true, y_pred, X,  top_n=10):

    errors = y_true - y_pred
    error_df = pd.DataFrame({'Error': errors, 'Actual': y_true, 'Predicted': y_pred})
    X_with_errors = pd.concat([X.reset_index(drop=True), error_df.reset_index(drop=True)], axis=1)

    # Display largest errors
    largest_errors = X_with_errors.nlargest(top_n, 'Error')
    print(f"\nTop {top_n} Largest Errors:")
    print(largest_errors)

    # Display smallest errors
    smallest_errors = X_with_errors.nsmallest(top_n, 'Error')
    print(f"\nTop {top_n} Smallest Errors:")
    print(smallest_errors)

    # Scatter plot of predicted vs actual, with errors color-coded
    plt.figure(figsize=(10, 8))
    plt.scatter(y_true, y_pred, c=errors, cmap='coolwarm', alpha=0.7)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title('Predicted vs. Actual Values with Errors')
    plt.colorbar(label='Error (Actual - Predicted)')
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--')  # Diagonal line
    plt.show()
    return X_with_errors

error_analysis_df = analyze_errors(y_test, predictions, X_test)

# ## 10. Further Steps (Indicating Continued Interest)

print("\nFurther Steps to Explore:")
print("- More extensive feature engineering based on molecular properties using RDKit (if available in the actual dataset).  Example:  calculate molecular weight, number of rings, etc.")
print("- Hyperparameter tuning of the individual models and the Voting Regressor using techniques like GridSearchCV or RandomizedSearchCV.  Example: tune the number of trees, learning rate, and max depth for Gradient Boosting.")
print("- Exploring more advanced ensemble methods like Stacking, where a meta-model learns how to combine the predictions of the base models.")
print("- Investigating the use of dimensionality reduction techniques (e.g., PCA, t-SNE) if the number of features is very high after feature engineering, to reduce complexity and potentially improve performance.")
print("- If molecular structure information is available (e.g., SMILES strings), using cheminformatics libraries like RDKit to generate molecular descriptors.  Example:  calculate more complex molecular descriptors and fingerprints.")
print("- Analyzing the types of errors made by the models to gain insights for improvement.  Are there specific types of molecules or specific ranges of coupling constants where the model performs poorly?")
print("- Experimenting with different ways of handling the molecular structure data, such as using graph neural networks (GNNs) to directly learn from the molecular graph representation.")
print("-  Consider using different cross-validation strategies, such as GroupKFold, if there are known dependencies between data points (e.g., if multiple rows come from the same molecule).")